<a href="https://colab.research.google.com/github/tadkollar/RevHack2020/blob/master/OpenVSPwithOpenMDAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating an OpenMDAO Component Wrapper for a VSP Model**

The OpenVSP installation here, specific to Colab, is adapted from the Ubuntu instructions provided on the [OpenVSP site](http://openvsp.org/wiki/doku.php?id=ubuntu_instructions).

First, install required packages:

In [1]:
!apt-get update && \
 apt-get install git cmake libxml2-dev libfltk1.3-dev g++ libcpptest-dev \
  libjpeg-dev libglm-dev libeigen3-dev libcminpack-dev libglew-dev swig \
  doxygen graphviz python3-venv

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security

Next, set up the directory structure and clone the OpenVSP source:

In [2]:
!mkdir -p repo build buildlibs && \
 git clone --depth=1 https://github.com/OpenVSP/OpenVSP.git repo

Cloning into 'repo'...
remote: Enumerating objects: 2194, done.
remote: Counting objects: 100% (2194/2194), done.
remote: Compressing objects: 100% (1623/1623), done.
remote: Total 2194 (delta 629), reused 1534 (delta 524), pack-reused 0
Receiving objects: 100% (2194/2194), 89.00 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (629/629), done.
Checking out files: 100% (2023/2023), done.


Prepare the build files for the libraries and build them:

In [3]:
!cd buildlibs && \
 cmake \
  -DVSP_USE_SYSTEM_LIBXML2=true \
  -DVSP_USE_SYSTEM_FLTK=true \
  -DVSP_USE_SYSTEM_GLM=true \
  -DVSP_USE_SYSTEM_GLEW=true \
  -DVSP_USE_SYSTEM_CMINPACK=true \
  -DVSP_USE_SYSTEM_LIBIGES=false \
  -DVSP_USE_SYSTEM_EIGEN=false \
  -DVSP_USE_SYSTEM_CODEELI=false \
  -DVSP_USE_SYSTEM_CPPTEST=false \
  ../repo/Libraries -DCMAKE_BUILD_TYPE=Release && \
 make -j 8

Streaming output truncated to the last 5000 lines.
[ 72%] Building CXX object src/exp2cxx/CMakeFiles/exp2cxx-static.dir/orlist.cc.o
[ 72%] Building CXX object src/clstepcore/CMakeFiles/stepcore.dir/attrDescriptorList.cc.o
[ 72%] Building CXX object src/clstepcore/CMakeFiles/stepcore.dir/collect.cc.o
[ 72%] Building CXX object src/clstepcore/CMakeFiles/stepcore.dir/complexlist.cc.o
[ 72%] Linking CXX executable ../../bin/exp2cxx
[ 73%] Building CXX object src/clstepcore/CMakeFiles/stepcore.dir/create_Aggr.cc.o
[ 73%] Building CXX object src/clstepcore/CMakeFiles/stepcore.dir/derivedAttribute.cc.o
[ 73%] Built target exp2cxx
Scanning dependencies of target generate_cpp_sdai_ap203
[ 73%] Building CXX object src/exp2cxx/CMakeFiles/exp2cxx-static.dir/entnode.cc.o
[ 74%] Building CXX object src/exp2cxx/CMakeFiles/exp2cxx-static.dir/expressbuild.cc.o
[ 74%] [exp2cxx] Generating 602 C++ files for sdai_ap203.
[ 74%] [exp2cxx] Generating 602 C++ files for sdai_ap203.
[ 74%] Building CXX object s

Build OpenVSP, which may take 10-15 minutes on Colab. Generate a zip folder that we can use to install the Python packages from:

In [4]:
!cd build && \
 cmake ../repo/src \
  -DVSP_LIBRARY_PATH=/content/buildlibs \
  -DCMAKE_BUILD_TYPE=Release && \
 make -j 8 && \
 make package

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Doxygen: /usr/bin/doxygen (found version "1.8.13") found components:  doxygen dot 
-- Found CPPTest: /content/buildlibs/CPPTEST-prefix/include  
-- Found LibXml2: /usr/lib/x86_64-linux-gnu/libxml2.so (found version "2.9.4") 
CMake Warning (dev) at /usr/local/lib/python2.7/dist-packages/cmake/data/share/cmake-3.12/Modules/FindOpenGL.cmake:270 (message):
  Policy CMP0072 is not set: FindOpenGL pr

Build and install the OpenVSP Python packages. The current version of OpenVSP may have to be updated:

In [5]:
import os
os.environ["VSP_VER"] = "3.21.2"

!cd /content/build/_CPack_Packages/Linux/ZIP/OpenVSP-${VSP_VER}-Linux/python && \
  pip install -r requirements.txt && \
  cd .. && \
  cp vspaero vspscript vspslicer /usr/local/bin

Processing ./utilities
Processing ./degen_geom
Processing ./openvsp
Processing ./CHARM
  Created wheel for utilities: filename=utilities-0.1.0-cp36-none-any.whl size=25295 sha256=96c4b173df169a84068264e18e5452b0ca79606ca4b2c7cb123a9f6114498c2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0eqesc3/wheels/ee/a9/e5/8d35b2640196c5575608cd62e8b044be6e6b9a0a2026406207
  Created wheel for degen-geom: filename=degen_geom-0.0.1-cp36-none-any.whl size=5210 sha256=d1b700eaed45fc6933504c5b7b8a739707f339e5336cdd15b1a0a1d2c25ff25a
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0eqesc3/wheels/a3/bb/32/d83f8e79012000ca9e9b7c66f4a36d71684c52499e14c3e88c
  Created wheel for openvsp: filename=openvsp-3.21.2-cp36-none-any.whl size=13544544 sha256=798236f1e0e1792035c77fee936fc3a73f93b0ed752c355346f2b97c5f424dba
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0eqesc3/wheels/2e/46/d6/5d6ce2679629b469c2e70cdf9d566c11e3c1fd0fef8b957433
  Created wheel for charm: filename=charm-0.1.0-cp36-none-any.w

Run a simple test to be sure the packages are installed correctly:

In [6]:
import openvsp

# **Incorporating OpenVSP into OpenMDAO**
## Install OpenMDAO


In [7]:
pip install openmdao jdc

     |████████████████████████████████| 4.6MB 2.7MB/s 
  Created wheel for openmdao: filename=openmdao-3.4.0-cp36-none-any.whl size=4821361 sha256=d075716c5e4cbf5b9a167a9f8e2e9e120acba7042215fc2674e29274edb5b7f5
  Stored in directory: /root/.cache/pip/wheels/72/ae/c6/95241129fd0f34471d0346d976a240dad8cf25282658bc05c1
  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-cp36-none-any.whl size=25519 sha256=cf3f9702eb3f679a7c7fdd9e0f962d44bad8f36be5d772d57daa0860368a86cb
  Stored in directory: /root/.cache/pip/wheels/60/88/1d/b334ee00dd83e82d111ec56c3be91573d335c93870698037f1
Successfully built openmdao pyDOE2


# Sample Code
This is an OpenMDAO component wrapper for a VSP model of a modified CRM (eCRM-001) that is used to demonstrate the computation and use of stability derivatives in a design problem. This code is based on a sample provided during OpenMDAO RevHack 2020.

First, obtain the CRM:

In [8]:
!wget -c https://raw.githubusercontent.com/OpenMDAO/RevHack2020/master/problems/oas_stability_derivs/eCRM-001.1_wing_tail.vsp3

--2020-10-29 18:22:17--  https://raw.githubusercontent.com/OpenMDAO/RevHack2020/master/problems/oas_stability_derivs/eCRM-001.1_wing_tail.vsp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394778 (386K) [text/plain]
Saving to: ‘eCRM-001.1_wing_tail.vsp3’

eCRM-001.1_wing_tai 100%[===================>] 385.53K  --.-KB/s    in 0.07s   

2020-10-29 18:22:17 (5.46 MB/s) - ‘eCRM-001.1_wing_tail.vsp3’ saved [394778/394778]



Import required modules:

In [9]:
import pickle
import itertools

import numpy as np

import openmdao.api as om

import openvsp as vsp
import degen_geom

import jdc # To permit class definitions to be spread across multiple cells

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


Create a new subclass based on OpenMDAO [ExplicitComponent](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/explicitcomp.html). Set some [options](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/options.html) with the names of the geometries in the VSP project that we want to work with. The reduced option is helpful for testing purposes.

In [10]:
class VSPeCRM(om.ExplicitComponent):

    def initialize(self):
        self.options.declare('horiz_tail_name', default='Tail',
                             desc="Name of the horizontal tail in the vsp model.")
        self.options.declare('vert_tail_name', default='VerticalTail',
                             desc="Name of the vertical tail in the vsp model.")
        self.options.declare('wing_name', default='Wing',
                             desc="Name of the wing in the vsp model.")
        self.options.declare('reduced', default=False,
                             desc="When True, output reduced meshes instead of full-size ones. "
                             "Running with a smaller mesh is of value when debugging.")

Load the OpenVSP project from a VSP3 file and find the IDs of the relevant geometries.

Set up inputs with initial values, and outputs with units and 3-dimensional shapes. The finite difference approximation method for partial derivatives is used.

In [11]:
    %%add_to VSPeCRM
    def setup(self):
        options = self.options
        horiz_tail_name = options['horiz_tail_name']
        vert_tail_name = options['vert_tail_name']
        wing_name = options['wing_name']
        reduced = options['reduced']

        # Read the geometry.
        vsp_file = 'eCRM-001.1_wing_tail.vsp3'
        vsp.ReadVSPFile(vsp_file)

        self.wing_id = vsp.FindGeomsWithName(wing_name)[0]
        self.horiz_tail_id = vsp.FindGeomsWithName(horiz_tail_name)[0]
        self.vert_tail_id = vsp.FindGeomsWithName(vert_tail_name)[0]

        self.add_input('wing_cord', val=59.05128,)
        self.add_input('vert_tail_area', val=2295.)
        self.add_input('horiz_tail_area', val=6336.)

        # Shapes are pre-determined.
        if reduced:
            self.add_output('wing_mesh', shape=(6, 9, 3), units='inch')
            self.add_output('vert_tail_mesh', shape=(5, 5, 3), units='inch')
            self.add_output('horiz_tail_mesh', shape=(5, 5, 3), units='inch')
        else:
            # Note: at present, OAS can't handle this size.
            self.add_output('wing_mesh', shape=(23, 33, 3), units='inch')
            self.add_output('vert_tail_mesh', shape=(33, 9, 3), units='inch')
            self.add_output('horiz_tail_mesh', shape=(33, 9, 3), units='inch')

        self.declare_partials(of='*', wrt='*', method='fd')

Compute the OpenVSP meshes. Using the previously located geometries, set the values of the VSP parameters to the initial input values from `setup()`. Then, reset the OpenVSP model to a blank slate with `Update()`.

Compute the degenerate geometry representation for the OpenVSP components, and obtain the measurements.

The purpose of this RevHack exercise was to use [OpenAeroStruct](https://github.com/mdolab/OpenAeroStruct), so update the meshes to work with it.

In [12]:
    %%add_to VSPeCRM
    def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):
        # set values
        vsp.SetParmVal(self.vert_tail_id, "TotalArea", "WingGeom", inputs['vert_tail_area'][0])
        vsp.SetParmVal(self.horiz_tail_id, "TotalArea", "WingGeom", inputs['horiz_tail_area'][0])
        vsp.SetParmVal(self.wing_id, "TotalChord", "WingGeom", inputs['wing_cord'][0])

        vsp.Update()
        vsp.Update()  # just in case..

        # run degen geom to get measurements
        dg:degen_geom.DegenGeomMgr = vsp.run_degen_geom(set_index=vsp.SET_ALL)
        obj_dict = {p.name:p for p in dg.get_all_objs()}

        # pull measurements out of degen_geom api
        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['wing_name']]
        wing_cloud = self.vsp_to_point_cloud(degen_obj)

        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['horiz_tail_name']]
        horiz_cloud = self.vsp_to_point_cloud(degen_obj)

        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['vert_tail_name']]
        vert_cloud = self.vsp_to_point_cloud(degen_obj)

        # VSP outputs wing outer mold lines at points along the span.
        # Reshape to (chord, span, dimension)
        wing_cloud = wing_cloud.reshape((45, 33, 3), order='F')
        horiz_cloud = horiz_cloud.reshape((33, 9, 3), order='F')
        vert_cloud = vert_cloud.reshape((33, 9, 3), order='F')

        # Meshes have upper and lower surfaces, so we average the z (or y for vertical).
        wing_pts = wing_cloud[:23, :, :]
        wing_pts[1:-1, :, 2] = 0.5 * (wing_cloud[-2:-23:-1, :, 2] + wing_pts[1:-1, :, 2])
        horiz_tail_pts = horiz_cloud[:17, :, :]
        horiz_tail_pts[1:-1, :, 2] = 0.5 * (horiz_cloud[-2:-17:-1, :, 2] + horiz_tail_pts[1:-1, :, 2])
        vert_tail_pts = vert_cloud[:17, :, :]
        vert_tail_pts[1:-1, :, 1] = 0.5 * (vert_cloud[-2:-17:-1, :, 1] + vert_tail_pts[1:-1, :, 1])

        # Reduce the mesh size for testing. (See John Jasa's recommendations in the docs.)
        if self.options['reduced']:
            wing_pts = wing_pts[:, ::4, :]
            wing_pts = wing_pts[[0, 4, 8, 12, 16, 22], ...]
            horiz_tail_pts = horiz_tail_pts[::4, ::2, :]
            vert_tail_pts = vert_tail_pts[::4, ::2, :]

        # Flip around so that FEM normals yield positive areas.
        wing_pts = wing_pts[::-1, ::-1, :]
        horiz_tail_pts = horiz_tail_pts[::-1, ::-1, :]
        vert_tail_pts = vert_tail_pts[::-1, ::-1, :]

        # outputs go here
        outputs['wing_mesh'] = wing_pts
        outputs['vert_tail_mesh'] = horiz_tail_pts
        outputs['horiz_tail_mesh'] = vert_tail_pts

Convert an OpenVSP degenerate geometry to a [NumPy N-dimensional array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) of points using [itertools](https://docs.python.org/3/library/itertools.html).

In [13]:
    %%add_to VSPeCRM
    def vsp_to_point_cloud(self, degen_obj: degen_geom.DegenGeom)->np.ndarray:
        npts = degen_obj.surf.num_pnts
        n_xsecs = degen_obj.surf.num_secs

        points = np.empty((npts * n_xsecs, 3))
        points[:, 0] = list(itertools.chain.from_iterable(degen_obj.surf.x))
        points[:, 1] = list(itertools.chain.from_iterable(degen_obj.surf.y))
        points[:, 2] = list(itertools.chain.from_iterable(degen_obj.surf.z))

        return points

Set up and run the model.


In [14]:
if __name__ == "__main__":

    vsp_comp = VSPeCRM(horiz_tail_name="Tail",
                       vert_tail_name="VerticalTail",
                       wing_name="Wing",
                       reduced=True)

    p = om.Problem()

    model = p.model

    p.model.add_subsystem("vsp_comp", vsp_comp)

    p.setup()

    p.run_model()

Serialize and save the data for later use in OpenAeroStruct.

In [15]:
    data = {}
    for item in ['wing_mesh', 'vert_tail_mesh', 'horiz_tail_mesh']:
        data[item] = p.get_val(f"vsp_comp.{item}", units='m')

    # Save the meshes in a pickle. These will become the undeformed baseline meshes in
    # OpenAeroStruct.
    with open('baseline_meshes_reduced.pkl', 'wb') as f:
        pickle.dump(data, f)

    print('done')

done
